In [1]:
import requests
import csv
import openpyxl
import xlrd

In [2]:
def getFootballUrl(league,season,format = "csv"):
    return f"https://www.football-data.co.uk/mmz4281/{season}/{league}.{format}"

def downloadData(league, season):
    season = f"{season}{season+1}"
    url = getFootballUrl(league, season)
    print(url)
    r = requests.get(url)
    if(r.status_code == 200):
        with open(f'../data/{league}-{season}.csv', 'wb') as f:
            f.write(r.content)
    else:
        print(f"{url}: File not found!")

def downloadAllEuropeanLeagues(startingYear):
    seasonShort = startingYear%100
    season = f"{seasonShort}{(seasonShort+1)%100}"
    ext = "xls"
    if startingYear >= 2017:
        ext = "xlsx"
    url = getFootballUrl(f"all-euro-data-{startingYear}-{startingYear+1}",season, ext)
    print(url)
    r = requests.get(url)
    firstCheck = 0 #TEMP
    if(r.status_code == 200):
        workbookPath = f'../data/AllLeagues-{season}.{ext}'
        print(f"Reading file: {workbookPath}")
        with open(workbookPath, 'wb') as f:
                f.write(r.content)
        if ext == "xlsx":
            book = openpyxl.load_workbook(filename = workbookPath, data_only=True)
            for leagueSheet in book:
                if leagueSheet.title == "G1": #Problem with greek league
                    print("League G1 has some problems and is not counted!")
                    continue
                print(f"Current league: {leagueSheet.title}")
                with open(f'../data/{leagueSheet.title}-{season}.csv', 'w') as csvFile:
                    wr = csv.writer(csvFile)
                    for row in leagueSheet.iter_rows(min_row = leagueSheet.min_row, max_row = leagueSheet.max_row, min_col = 0, max_col = 68):
                        rowData = []
                        for cell in row:
                            rowData.append(cell.value)
                        wr.writerow(rowData)
        else:
            book = xlrd.open_workbook(workbookPath)
            for leagueName in book.sheet_names():
                if leagueName == "G1":
                    print("League G1 has some problems and is not counted!")
                    continue
                print(f"Current league: {leagueName}")
                sh = book.sheet_by_name(leagueName)
                with open(f'../data/{leagueName}-{season}.csv', 'w') as csvFile:
                    wr = csv.writer(csvFile)
                    for rownum in range(sh.nrows):
                        wr.writerow(sh.row_values(rownum))

    else:
        print(f"Invalid URL: {url}")

In [8]:
downloadAllEuropeanLeagues(2012)

https://www.football-data.co.uk/mmz4281/1314/all-euro-data-2013-2014.xls
Reading file: ../data/AllLeagues-1314.xls
Current league: E0
Current league: E1
Current league: E2
Current league: E3
Current league: EC
Current league: SC0
Current league: SC1
Current league: SC2
Current league: SC3
Current league: D1
Current league: D2
Current league: SP1
Current league: SP2
Current league: I1
Current league: I2
Current league: F1
Current league: F2
Current league: B1
Current league: N1
Current league: P1
Current league: T1
League G1 has some problems and is not counted!


In [3]:
downloadData('E0',19)

https://www.football-data.co.uk/mmz4281/1920/E0.csv


In [24]:
with open("../data/E0-1415.csv") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        print(len(row))

68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
68
6